# Create variables

In [2]:
import numpy as np
import pandas as pd
import os
import pickle
from joblib import Parallel, delayed
import re
import matplotlib.pyplot as plt
import seaborn as sns

d = './raw_data/'

In [5]:
with open('processed_data/clean_build.pkl', 'rb') as filename:  
    clean_data = pickle.load(filename)

/var/folders/_q/50__mrnj2zv6hb8crhk3q46m0000gn/T/ipykernel_14611/3513447208.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  clean_data = pickle.load(filename)


In [9]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6822 entries, 0 to 6821
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     6822 non-null   object 
 1   budget                    6822 non-null   float64
 2   worldwide_boxoffice       6822 non-null   float64
 3   domestic_boxoffice        6822 non-null   float64
 4   clean_title               6822 non-null   object 
 5   imdb_id                   6812 non-null   object 
 6   release_date              6766 non-null   object 
 7   year                      6766 non-null   float64
 8   top_director              6822 non-null   object 
 9   top_cast                  6822 non-null   object 
 10  rating                    6822 non-null   float64
 11  runtime                   6822 non-null   float64
 12  original_language         6822 non-null   object 
 13  enrichment_source         6822 non-null   object 
 14  boxoffic

In [16]:
clean_data.top_cast

0       ['Zooey Deschanel', 'Joseph Gordon-Levitt', 'G...
1       ['Steven Strait', 'Camilla Belle', 'Cliff Curt...
2       ['Glenn Close', 'Jeff Daniels', 'Joely Richard...
3       ['Glenn Close', 'Gérard Depardieu', 'Ioan Gruf...
4       ['John Goodman', 'Mary Elizabeth Winstead', 'J...
                              ...                        
6817    ['Kim Yu-seok', 'Suh Jung', 'Seo Won', 'Son Mi...
6818    ['Kim Min-hee', 'Kim Tae-ri', 'Ha Jung-woo', '...
6819    ['Choi Min-sik', 'Yoo Ji-tae', 'Kang Hye-jung'...
6820    ['Park Hae-il', 'Moon Chae-won', 'Kim Moo-yul'...
6821    ['Cho Min-soo', 'Lee Jung-jin', 'Woo Ki-hong',...
Name: top_cast, Length: 6822, dtype: object

In [11]:
#function to adjust for inflation, input the number and year and it will say the value adjusted for inflation to 2025
def adjust_inflation(num, year):
    diff = 2025 - year
    final_num = num * (1.0325)**diff
    return final_num
    
# def get_entry(x, entry = 'name', rank = 0):
#     if isinstance(x, list):
#         entries = [i[entry] for i in x]
#         # keep only top 3 entries
#         if len(entries) > rank:
#             out = entries[rank]
#         else:
#             out = np.nan
#         return out

#     return np.nan

def get_entry(x, rank = 0):
    if isinstance(x, list) and len(x) > rank:
        out = entries[rank]
    else:
        out = np.nan
    return out

    return np.nan

def value_before_year(x, variable = "actor_id_1", category = "top_cast", value_field = "wroi", period = 5, data = clean_data):

    use_data = data.loc[(x['year'] > data['year'])*((x['year'] - period) <= data['year'])]
    if len(use_data) > 0:
        mask = np.array([x[variable] in entry for entry in use_data[category]]) if variable != 'director_id' else np.array([x[variable] == entry for entry in use_data[category]])
        if np.sum(mask) > 0:
            vallist = use_data[mask]
            value = vallist[value_field].mean()
        else:
            value = 0
    else:
        value = 0
    return value

In [13]:
# # Create adjusted revenue variables
clean_data.loc[:, 'adjusted_worldwide_boxoffice'] = adjust_inflation(clean_data['worldwide_boxoffice'], clean_data['year'])
clean_data.loc[:, 'adjusted_domestic_boxoffice'] = adjust_inflation(clean_data['domestic_boxoffice'], clean_data['year'])
clean_data.loc[:, 'adjusted_budget'] = adjust_inflation(clean_data['budget'], clean_data['year'])
clean_data.loc[:,'wroi'] = clean_data['adjusted_worldwide_boxoffice']/clean_data['adjusted_budget']
clean_data.loc[:,'droi'] = clean_data['adjusted_domestic_boxoffice']/clean_data['adjusted_budget']


for i in np.arange(0,3):
    clean_data.loc[:, 'actor_name_'+str(i + 1)] = clean_data['top_cast'].apply(get_entry, args = ([i]))
    clean_data.loc[:, 'actor_id_'+str(i + 1)] = clean_data['top_cast'].apply(get_entry, args = ([i]))
    clean_data.loc[:, 'genre_name_'+str(i + 1)] = clean_data['genres_list'].apply(get_entry, args = ([i]))
    clean_data.loc[:, 'production_country_'+str(i + 1)] = clean_data['top_production_countries'].apply(get_entry, args = ([i]))
    clean_data.loc[:, 'production_company_'+str(i + 1)] = clean_data['top_production_companies'].apply(get_entry, args = ([i]))

In [17]:
clean_data.actor_id_1

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
6817   NaN
6818   NaN
6819   NaN
6820   NaN
6821   NaN
Name: actor_id_1, Length: 6822, dtype: float64

In [ ]:
pairs = [['director_id', 'director_id', 'wroi'],
         ["actor_id_1", "top_cast_id", 'wroi'],["actor_id_2", "top_cast_id", 'wroi'], ["actor_id_3", "top_cast_id", 'wroi'],
         ["genre_name_1", "top_genres", 'wroi'],["genre_name_2", "top_genres", 'wroi'], ["genre_name_2", "top_genres", 'wroi'],
         ["production_company_1", "top_production_companies", 'wroi'],["production_company_1", "top_production_companies", 'wroi'], ["production_company_1", "top_production_companies", 'wroi']]

for p in pairs:
    combined_dataset.loc[:, 'avg_'+p[2]+'_'+p[0]] = combined_dataset.apply(value_before_year, args = ([p[0], p[1], p[2]]), axis = 1)